In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import panel as pn

pn.extension("ipywidgets", "katex", "mathjax")
import sys
from inspect import signature
from random import shuffle, uniform

import ipywidgets as ipw
from matplotlib.animation import FuncAnimation
from matplotlib.figure import Figure

print("Packages successfully loaded")

# Cookbook

In [ ]:
try:
    %run initialize/init_cookbook.ipynb # valid when running the cookbook in the main file
except:
    %run init_cookbook.ipynb # valid when running the cookbook from this file.

# Commonly used functions

In [ ]:
def wave_length(T, h):
    d = h

    # based on waveNumber_Fenton(T,d) from Jaime in computerlab
    g = 9.81
    omega = 2 * np.pi / T
    k0 = omega * omega / g
    alpha = k0 * d
    beta = alpha * (np.tanh(alpha)) ** -0.5
    k = (
        (alpha + beta**2 * np.cosh(beta) ** -2)
        / (np.tanh(beta) + beta * np.cosh(beta) ** -2)
        / d
    )

    L = 2 * np.pi / k

    return L


def group_stats(k1, k2, w1, w2):
    Delta_k = np.abs(k2 - k1)
    Delta_w = np.abs(w2 - w1)
    L = 2 * np.pi / Delta_k
    T = 2 * np.pi / Delta_w
    cg = Delta_w / Delta_k
    return L, T, cg

# Q1 surface coding

In [ ]:
def Show_W4_Q1():
    # define the name of the function that the students will make
    function_name = "envelope_eta_t"

    # define the name of the parameter plotted on the horizontal axis
    parameter_x_axis = "t"

    # Setup structure for multiple plots
    fig, axs = plt.subplots(
        nrows=1, ncols=2, figsize=(10, 3), sharex=False, sharey=True
    )
    pane = pn.pane.Matplotlib(fig, dpi=100)
    fig.subplots_adjust(wspace=0)  # no horizontal space between plots

    # set the horizontal axis of the graph
    L1 = wave_length(T1, h)
    L2 = wave_length(T2, h)
    w1 = 2 * np.pi / T1
    w2 = 2 * np.pi / T2
    k1 = 2 * np.pi / L1
    k2 = 2 * np.pi / L2

    L_group, T_group, c_group = group_stats(k1, k2, w1, w2)
    Delta_T = np.min([T1, T2]) / 20
    Delta_x = np.min([L1, L2]) / 20
    t_max = 3 * T_group
    x_max = 3 * L_group

    horizontal_axis1 = np.arange(0, t_max + Delta_T, Delta_T)
    horizontal_axis2 = np.arange(0, x_max + Delta_x, Delta_x)

    # define the correct function
    def correct_function(a1, a2, T1, T2, t, xp):
        L1 = wave_length(T1, h)
        L2 = wave_length(T2, h)

        # eta = 2a cos( Delta omega/2  t- Delta k/2 x)
        w1 = 2 * np.pi / T1
        w2 = 2 * np.pi / T2
        k1 = 2 * np.pi / L1
        k2 = 2 * np.pi / L2
        Delta_w = np.abs(w1 - w2)
        Delta_k = k2 - k1
        a = np.abs((a1 + a2) * np.cos(0.5 * Delta_w * t - 0.5 * Delta_k * xp))
        return a

    # set the acceptable computational error (ratio)
    f_margin = 0.1  # 0.1 = 1%

    ax = axs[0]
    pane, ax = check_code_function(
        fig,
        horizontal_axis1,
        function_name,
        correct_function,
        parameter_x_axis,
        f_margin,
        new_graph=False,
        ax=ax,
        pane=pane,
        xlabel="time [s]",
        ylabel="y coordinate [m]",
    )

    function_name = "envelope_eta_x"
    parameter_x_axis = "x"

    def correct_function(a1, a2, T1, T2, tp, x):
        L1 = wave_length(T1, h)
        L2 = wave_length(T2, h)

        # eta = 2a cos( Delta omega/2  t- Delta k/2 x)
        w1 = 2 * np.pi / T1
        w2 = 2 * np.pi / T2
        k1 = 2 * np.pi / L1
        k2 = 2 * np.pi / L2
        Delta_w = np.abs(w1 - w2)
        Delta_k = k2 - k1
        a = np.abs((a1 + a2) * np.cos(0.5 * Delta_w * tp - 0.5 * Delta_k * x))
        return a

    ax = axs[1]
    pane, ax = check_code_function(
        fig,
        horizontal_axis2,
        function_name,
        correct_function,
        parameter_x_axis,
        f_margin,
        new_graph=False,
        ax=ax,
        pane=pane,
        xlabel="space [x]",
    )

    # Plot the wave components if a line is plotted.
    if len(axs[0].lines) > 0:
        t = horizontal_axis1
        eta1_T = a1 * np.sin(2 * np.pi / T1 * t - 2 * np.pi / L1 * xp)
        eta2_T = a2 * np.sin(2 * np.pi / T2 * t - 2 * np.pi / L2 * xp)
        eta_T = eta1_T + eta2_T

        axs[0].plot(t, eta_T, label="$\eta$")
        axs[0].legend(bbox_to_anchor=(-0.02, 1), loc="lower left")  # update the legend

    if len(axs[1].lines) > 0:
        x = horizontal_axis2
        eta1_X = a1 * np.sin(2 * np.pi / T1 * tp - 2 * np.pi / L1 * x)
        eta2_X = a2 * np.sin(2 * np.pi / T2 * tp - 2 * np.pi / L2 * x)
        eta_X = eta1_X + eta2_X

        axs[1].plot(x, eta_X, label="$\eta$")
        axs[1].legend(bbox_to_anchor=(-0.02, 1), loc="lower left")  # update the legend

## Wave energy coding

In [ ]:
def Show_W4_Q2():
    # define the name of the function that the students will make
    function_name = "envelope_E_t"

    # define the name of the parameter plotted on the horizontal axis
    parameter_x_axis = "t"

    # Setup structure for multiple plots
    fig, axs = plt.subplots(
        nrows=2, ncols=2, figsize=(10, 4), sharex=False, sharey=False
    )
    pane = pn.pane.Matplotlib(fig, dpi=100)
    fig.subplots_adjust(
        wspace=0, hspace=0
    )  # no horizontal and small vertical space between plots
    axs[0, 0].tick_params(
        axis="x", which="both", bottom=False, top=False, labelbottom=False
    )  # no x-axis on top row, first column
    axs[0, 1].tick_params(
        axis="both",
        which="both",
        bottom=False,
        top=False,
        left=False,
        right=False,
        labelbottom=False,
        labelleft=False,
    )  # no xlabel and ylabels on top row, second (left) column.
    axs[1, 1].tick_params(axis="y", which="both", left=False, labelleft=False)
    axs[0, 0].set_ylabel("y-coordinate [m]")

    # set the horizontal axis of the graph
    L1 = wave_length(T1, h)
    L2 = wave_length(T2, h)
    w1 = 2 * np.pi / T1
    w2 = 2 * np.pi / T2
    k1 = 2 * np.pi / L1
    k2 = 2 * np.pi / L2

    L_group, T_group, c_group = group_stats(k1, k2, w1, w2)
    Delta_T = np.min([T1, T2]) / 20
    Delta_x = np.min([L1, L2]) / 20
    t_max = 3 * T_group
    x_max = 3 * L_group

    horizontal_axis1 = np.arange(0, t_max + Delta_T, Delta_T)
    horizontal_axis2 = np.arange(0, x_max + Delta_x, Delta_x)

    # define the correct function
    rho = 1025

    def correct_function(a1, a2, T1, T2, t, xp, rho):
        L1 = wave_length(T1, h)
        L2 = wave_length(T2, h)

        # eta = 2a cos( Delta omega/2  t- Delta k/2 x)
        w1 = 2 * np.pi / T1
        w2 = 2 * np.pi / T2
        k1 = 2 * np.pi / L1
        k2 = 2 * np.pi / L2
        Delta_w = np.abs(w1 - w2)
        Delta_k = k2 - k1
        eta = (a1 + a2) * np.cos(0.5 * Delta_w * t - 0.5 * Delta_k * xp)
        g = 9.81  # m/s^2
        E = 1 / 2 * rho * g * eta**2  # The wave energy
        return E

    # set the acceptable computational error (ratio)
    f_margin = 0.1  # 0.1 = 1%

    ax = axs[1, 0]
    pane, ax = check_code_function(
        fig,
        horizontal_axis1,
        function_name,
        correct_function,
        parameter_x_axis,
        f_margin,
        new_graph=False,
        ax=ax,
        pane=pane,
        xlabel="time [s]",
        ylabel="E $[m^2]$",
    )
    # adjust the location of the legend
    if len(ax.lines) > 0:
        ax.legend(loc="upper right")

    function_name = "envelope_E_x"
    parameter_x_axis = "x"

    def correct_function(a1, a2, T1, T2, tp, x, rho):
        L1 = wave_length(T1, h)
        L2 = wave_length(T2, h)

        # eta = 2a cos( Delta omega/2  t- Delta k/2 x)
        w1 = 2 * np.pi / T1
        w2 = 2 * np.pi / T2
        k1 = 2 * np.pi / L1
        k2 = 2 * np.pi / L2
        Delta_w = np.abs(w1 - w2)
        Delta_k = k2 - k1
        eta = (a1 + a2) * np.cos(0.5 * Delta_w * tp - 0.5 * Delta_k * x)
        g = 9.81  # m/s^2
        E = 1 / 2 * rho * g * eta**2  # The wave energy
        return E

    ax = axs[1, 1]
    pane, ax = check_code_function(
        fig,
        horizontal_axis2,
        function_name,
        correct_function,
        parameter_x_axis,
        f_margin,
        new_graph=False,
        ax=ax,
        pane=pane,
        xlabel="space [x]",
    )
    # adjust the location of the legend
    if len(ax.lines) > 0:
        ax.legend(loc="upper right")

    # Plot the wave components on the top row
    t = horizontal_axis1
    eta1_T = a1 * np.sin(2 * np.pi / T1 * t - 2 * np.pi / L1 * xp)
    eta2_T = a2 * np.sin(2 * np.pi / T2 * t - 2 * np.pi / L2 * xp)
    eta_T = eta1_T + eta2_T

    axs[0, 0].plot(t, eta_T, label="$\eta$")
    axs[0, 0].legend(loc="upper right")

    x = horizontal_axis2
    eta1_X = a1 * np.sin(2 * np.pi / T1 * tp - 2 * np.pi / L1 * x)
    eta2_X = a2 * np.sin(2 * np.pi / T2 * tp - 2 * np.pi / L2 * x)
    eta_X = eta1_X + eta2_X

    axs[0, 1].plot(x, eta_X, label="$\eta$")
    axs[0, 1].legend(loc="upper right")

## Sxx coding

In [ ]:
def Show_W4_Q3():
    # define the name of the function that the students will make
    function_name = "envelope_Sxx_t"

    # define the name of the parameter plotted on the horizontal axis
    parameter_x_axis = "t"

    # Setup structure for multiple plots
    fig, axs = plt.subplots(
        nrows=2, ncols=2, figsize=(10, 4), sharex=False, sharey=False
    )
    pane = pn.pane.Matplotlib(fig, dpi=100)
    fig.subplots_adjust(
        wspace=0, hspace=0
    )  # no horizontal and small vertical space between plots
    axs[0, 0].tick_params(
        axis="x", which="both", bottom=False, top=False, labelbottom=False
    )  # no x-axis on top row, first column
    axs[0, 1].tick_params(
        axis="both",
        which="both",
        bottom=False,
        top=False,
        left=False,
        right=False,
        labelbottom=False,
        labelleft=False,
    )  # no xlabel and ylabels on top row, second (left) column.
    axs[1, 1].tick_params(axis="y", which="both", left=False, labelleft=False)
    axs[0, 0].set_ylabel("y-coordinate [m]")

    # set the horizontal axis of the graph
    L1 = wave_length(T1, h)
    L2 = wave_length(T2, h)
    w1 = 2 * np.pi / T1
    w2 = 2 * np.pi / T2
    k1 = 2 * np.pi / L1
    k2 = 2 * np.pi / L2

    L_group, T_group, c_group = group_stats(k1, k2, w1, w2)
    Delta_T = np.min([T1, T2]) / 20
    Delta_x = np.min([L1, L2]) / 20
    t_max = 3 * T_group
    x_max = 3 * L_group

    horizontal_axis1 = np.arange(0, t_max + Delta_T, Delta_T)
    horizontal_axis2 = np.arange(0, x_max + Delta_x, Delta_x)

    # define the correct function
    rho = 1025

    def correct_function(a1, a2, T1, T2, t, xp, rho):
        L1 = wave_length(T1, h)
        L2 = wave_length(T2, h)

        # eta = 2a cos( Delta omega/2  t- Delta k/2 x)
        w1 = 2 * np.pi / T1
        w2 = 2 * np.pi / T2
        k1 = 2 * np.pi / L1
        k2 = 2 * np.pi / L2
        Delta_w = np.abs(w1 - w2)
        Delta_k = k2 - k1
        eta = (a1 + a2) * np.cos(0.5 * Delta_w * t - 0.5 * Delta_k * xp)
        g = 9.81  # m/s^2
        E = 1 / 2 * rho * g * eta**2  # The wave energy

        c1 = L1 / T1
        c2 = L2 / T2
        c_average = (c1 + c2) / 2

        w1 = 2 * np.pi / T1
        w2 = 2 * np.pi / T2
        k1 = 2 * np.pi / L1
        k2 = 2 * np.pi / L2
        Delta_omega = np.abs(w1 - w2)
        Delta_k = np.abs(k1 - k2)
        cg = Delta_omega / Delta_k
        n = cg / c_average

        Sxx = (2 * n - 0.5) * E

        return Sxx

    # set the acceptable computational error (ratio)
    f_margin = 0.1  # 0.1 = 1%

    ax = axs[1, 0]
    pane, ax = check_code_function(
        fig,
        horizontal_axis1,
        function_name,
        correct_function,
        parameter_x_axis,
        f_margin,
        new_graph=False,
        ax=ax,
        pane=pane,
        xlabel="time [s]",
        ylabel="Sxx $[m^2]$",
    )
    # adjust the location of the legend
    if len(ax.lines) > 0:
        ax.legend(loc="upper right")

    function_name = "envelope_Sxx_x"
    parameter_x_axis = "x"

    def correct_function(a1, a2, T1, T2, tp, x, rho):
        L1 = wave_length(T1, h)
        L2 = wave_length(T2, h)

        # eta = 2a cos( Delta omega/2  t- Delta k/2 x)
        w1 = 2 * np.pi / T1
        w2 = 2 * np.pi / T2
        k1 = 2 * np.pi / L1
        k2 = 2 * np.pi / L2
        Delta_w = np.abs(w1 - w2)
        Delta_k = k2 - k1
        eta = (a1 + a2) * np.cos(0.5 * Delta_w * tp - 0.5 * Delta_k * x)
        g = 9.81  # m/s^2
        E = 1 / 2 * rho * g * eta**2  # The wave energy

        c1 = L1 / T1
        c2 = L2 / T2
        c_average = (c1 + c2) / 2

        w1 = 2 * np.pi / T1
        w2 = 2 * np.pi / T2
        k1 = 2 * np.pi / L1
        k2 = 2 * np.pi / L2
        Delta_omega = np.abs(w1 - w2)
        Delta_k = np.abs(k1 - k2)
        cg = Delta_omega / Delta_k
        n = cg / c_average

        Sxx = (2 * n - 0.5) * E
        return Sxx

    ax = axs[1, 1]
    pane, ax = check_code_function(
        fig,
        horizontal_axis2,
        function_name,
        correct_function,
        parameter_x_axis,
        f_margin,
        new_graph=False,
        ax=ax,
        pane=pane,
        xlabel="space [x]",
    )
    # adjust the location of the legend
    if len(ax.lines) > 0:
        ax.legend(loc="upper right")

    # Plot the wave components on the top row
    t = horizontal_axis1
    eta1_T = a1 * np.sin(2 * np.pi / T1 * t - 2 * np.pi / L1 * xp)
    eta2_T = a2 * np.sin(2 * np.pi / T2 * t - 2 * np.pi / L2 * xp)
    eta_T = eta1_T + eta2_T

    axs[0, 0].plot(t, eta_T, label="$\eta$")
    axs[0, 0].legend(loc="upper right")

    x = horizontal_axis2
    eta1_X = a1 * np.sin(2 * np.pi / T1 * tp - 2 * np.pi / L1 * x)
    eta2_X = a2 * np.sin(2 * np.pi / T2 * tp - 2 * np.pi / L2 * x)
    eta_X = eta1_X + eta2_X

    axs[0, 1].plot(x, eta_X, label="$\eta$")
    axs[0, 1].legend(loc="upper right")

## eta_b

### Magnitude amplitude

In [ ]:
def init_W4_Q4():
    # The calculation

    # Required widgets for functionality, do not have to be changed
    attempt = pn.widgets.FloatInput(value=0)

    # define a new global variable, so return (and related print) is prevented,
    # + a required parameter to count the attempt
    global W4_Q4_param
    # store the question-related parameters and
    W4_Q4_param = attempt


# The function parameters and will be loaded into the students notebook when the %run is executed.
init_W4_Q4()

In [ ]:
def Check_W4_Q4():
    attempt = W4_Q4_param

    L1 = wave_length(T1, h)
    L2 = wave_length(T2, h)

    # eta = 2a cos( Delta omega/2  t- Delta k/2 x)
    w1 = 2 * np.pi / T1
    w2 = 2 * np.pi / T2
    k1 = 2 * np.pi / L1
    k2 = 2 * np.pi / L2
    Delta_w = np.abs(w1 - w2)
    Delta_k = k2 - k1
    eta = (a1 + a2) * np.cos(0.5 * Delta_w * t - 0.5 * Delta_k * xp)
    g = 9.81  # m/s^2
    E = 1 / 8 * rho * g * eta**2  # The wave energy

    c1 = L1 / T1
    c2 = L2 / T2
    c_average = (c1 + c2) / 2

    w1 = 2 * np.pi / T1
    w2 = 2 * np.pi / T2
    k1 = 2 * np.pi / L1
    k2 = 2 * np.pi / L2
    Delta_omega = np.abs(w1 - w2)
    Delta_k = np.abs(k1 - k2)
    cg = Delta_omega / Delta_k
    n = cg / c_average

    g = 9.81
    a = a1
    eta_b = g * a**2 * (2 * n - 0.5) / (g * h - cg**2)

    # define the parameter names that have to be checked
    check_parameters = ["eta_b"]

    # define the names of the parameters as they are displayed
    name_parameters = ["eta b"]

    # additional settings
    n_decimals = 4  # decimals of the answer shown when 3 wrong answers
    f_margin = 0.1

    # build  the question
    FV = classify_variables(locals())
    check_code_values(FV)

### Coding

In [ ]:
def Show_W4_Q4():
    # define the name of the function that the students will make
    function_name = "boundwave_eta_t"

    # define the name of the parameter plotted on the horizontal axis
    parameter_x_axis = "t"

    # Setup structure for multiple plots
    fig, axs = plt.subplots(
        nrows=1, ncols=2, figsize=(10, 3), sharex=False, sharey=True
    )
    pane = pn.pane.Matplotlib(fig, dpi=100)
    fig.subplots_adjust(wspace=0)  # no horizontal space between plots

    # set the horizontal axis of the graph
    L1 = wave_length(T1, h)
    L2 = wave_length(T2, h)
    w1 = 2 * np.pi / T1
    w2 = 2 * np.pi / T2
    k1 = 2 * np.pi / L1
    k2 = 2 * np.pi / L2

    L_group, T_group, c_group = group_stats(k1, k2, w1, w2)
    Delta_T = np.min([T1, T2]) / 20
    Delta_x = np.min([L1, L2]) / 20
    t_max = 3 * T_group
    x_max = 3 * L_group

    horizontal_axis1 = np.arange(0, t_max + Delta_T, Delta_T)
    horizontal_axis2 = np.arange(0, x_max + Delta_x, Delta_x)

    # define the correct function
    rho = 1025

    def correct_function(a1, a2, T1, T2, t, xp, rho):
        g = 9.81  # m/s^2

        L1 = wave_length(T1, h)
        L2 = wave_length(T2, h)

        w1 = 2 * np.pi / T1
        w2 = 2 * np.pi / T2
        k1 = 2 * np.pi / L1
        k2 = 2 * np.pi / L2

        Delta_w = np.abs(w1 - w2)
        Delta_k = k2 - k1

        c1 = L1 / T1
        c2 = L2 / T2
        c_average = (c1 + c2) / 2

        Delta_omega = np.abs(w1 - w2)
        Delta_k = np.abs(k1 - k2)
        cg = Delta_omega / Delta_k
        n = cg / c_average

        g = 9.81
        a = a1
        eta_b = g * a**2 * (2 * n - 0.5) / (g * h - cg**2)
        eta = eta_b * np.cos(0.5 * Delta_omega * t - 0.5 * Delta_k * xp)

        return eta

    # set the acceptable computational error (ratio)
    f_margin = 0.1  # 0.1 = 1%

    ax = axs[0]
    pane, ax = check_code_function(
        fig,
        horizontal_axis1,
        function_name,
        correct_function,
        parameter_x_axis,
        f_margin,
        new_graph=False,
        ax=ax,
        pane=pane,
        xlabel="time [s]",
        ylabel="y coordinate [m]",
    )

    function_name = "boundwave_eta_x"
    parameter_x_axis = "x"

    def correct_function(a1, a2, T1, T2, tp, x, rho):
        g = 9.81  # m/s^2

        L1 = wave_length(T1, h)
        L2 = wave_length(T2, h)

        w1 = 2 * np.pi / T1
        w2 = 2 * np.pi / T2
        k1 = 2 * np.pi / L1
        k2 = 2 * np.pi / L2

        Delta_w = np.abs(w1 - w2)
        Delta_k = k2 - k1

        c1 = L1 / T1
        c2 = L2 / T2
        c_average = (c1 + c2) / 2

        Delta_omega = np.abs(w1 - w2)
        Delta_k = np.abs(k1 - k2)
        cg = Delta_omega / Delta_k
        n = cg / c_average

        g = 9.81
        a = a1
        eta_b = g * a**2 * (2 * n - 0.5) / (g * h - cg**2)
        eta = eta_b * np.cos(0.5 * Delta_omega * tp - 0.5 * Delta_k * x)
        return eta

    ax = axs[1]
    pane, ax = check_code_function(
        fig,
        horizontal_axis2,
        function_name,
        correct_function,
        parameter_x_axis,
        f_margin,
        new_graph=False,
        ax=ax,
        pane=pane,
        xlabel="space [x]",
    )

    # Plot the wave components and envelope if a line is plotted.
    L1 = wave_length(T1, h)
    L2 = wave_length(T2, h)
    w1 = 2 * np.pi / T1
    w2 = 2 * np.pi / T2
    k1 = 2 * np.pi / L1
    k2 = 2 * np.pi / L2
    Delta_w = np.abs(w1 - w2)
    Delta_k = k2 - k1

    if len(axs[0].lines) > 0:
        t = horizontal_axis1
        eta1_T = a1 * np.sin(2 * np.pi / T1 * t - 2 * np.pi / L1 * xp)
        eta2_T = a2 * np.sin(2 * np.pi / T2 * t - 2 * np.pi / L2 * xp)
        eta_T = eta1_T + eta2_T

        a_T = np.abs((a1 + a2) * np.cos(0.5 * Delta_w * t - 0.5 * Delta_k * xp))

        axs[0].plot(t, eta_T, label="$\eta$")
        axs[0].plot(t, a_T, label="envelope")
        axs[0].legend(bbox_to_anchor=(-0.02, 1), loc="lower left")  # update the legend

    if len(axs[1].lines) > 0:
        x = horizontal_axis2
        eta1_X = a1 * np.sin(2 * np.pi / T1 * tp - 2 * np.pi / L1 * x)
        eta2_X = a2 * np.sin(2 * np.pi / T2 * tp - 2 * np.pi / L2 * x)
        eta_X = eta1_X + eta2_X

        a_X = np.abs((a1 + a2) * np.cos(0.5 * Delta_w * tp - 0.5 * Delta_k * x))

        axs[1].plot(x, eta_X, label="$\eta$")
        axs[1].plot(x, a_X, label="envelope")
        axs[1].legend(bbox_to_anchor=(-0.02, 1), loc="lower left")  # update the legend